# In-Class Assignment — Data Preprocessing & Cleaning (Text)  
**Time:** 20 minutes  |  **Points:** 10  

## Instructions
- This is an individual in-class assignment.  
- Write your code **inside each answer cell**.  
- Print the required outputs.  
- Submit your GitHub/Colab link as instructed by the instructor.


You are given a small dataset of customer support messages as a **TAB-separated text file**:  
- `support_messages.txt`

You will download this file from **Canvas** and upload it to your **Google Colab** notebook.

**How to upload it to your Google Colab notebook?**

1. Download `support_messages.txt` from Canvas.
3. In **the left sidebar**, click the **Files** icon (folder).  
4. Click **Upload** and select `support_messages.txt`.

6. RightAfter uploading, the file will appear in the Colab file list on the left.

6. Right-click the file, copy its path, and paste it into the FILE_PATH variable in Q1.

7. Run Q1 to load the dataset.



> Important: Keep the file name exactly as `support_messages.txt`.


## Questions (Total = 10 points)

### Q1 (1 point) — Load the dataset
Load the TAB-separated file into a pandas DataFrame with columns: `id`, `message`.  
Print: **(a)** `df.shape`, **(b)** `df.head(3)`.

### Q2 (3 points) — Descriptive columns
Add these columns for each message and print the full DataFrame:
- `word_count`: number of words  
- `char_count`: number of characters  
- `num_count`: number of digits (0–9)  
- `upper_word_count`: number of ALL-CAPS words (e.g., `"WHY"`, `"DAMAGED"`)  

### Q3 (3 points) — Clean text
Build a `clean_text(text)` function and create a new column `clean` with these steps **in order**:
1) lowercase  
2) remove punctuation/symbols (keep letters/numbers/spaces)  
3) remove English stopwords (use **nltk** or **sklearn** list)  
4) remove extra spaces  

Print the **original** message and **clean** version for rows `id=1` and `id=4`.

### Q4 (2 points) — Regex extraction
Using RegEx, extract and create two new columns:
- `order_id`: first occurrence of pattern `ORD-####` (case-insensitive; `ord-1060` is valid)  
- `email`: first email address if present (otherwise `None`/`NaN`)  

Print: `id`, `order_id`, `email` for all rows.

### Q5 (1 point) — TF-IDF keywords
Using the `clean` column, compute **TF-IDF** for the messages and print the **top 5 keywords** with the highest **average TF-IDF** across documents.


In [1]:
# Setup (run this cell first)
import re
import pandas as pd


## Q1 (1 point) — Answer below

In [8]:
# Q1 — ANSWER CELL
FILE_PATH = "/content/support_messages.txt"  # ← Replace with your copied path

# Load TAB-separated file
# df = pd.read_csv(FILE_PATH, sep="\t", names=["id", "message"])
df = pd.read_csv(FILE_PATH, sep="\t")

# Print required outputs
print("Shape:", df.shape)
print("\nFirst 3 rows:")
print(df.head(3))

Shape: (8, 2)

First 3 rows:
   id                                            message
0   1  Hi!! My ORDER is late :(  Order# ORD-1042. Ema...
1   2  Refund please!!! I was charged 2 times... invo...
2   3        Great service, thanks! arrived in 2 days :)


## Q2 (3 points) — Answer below

In [9]:
# Q2 — ANSWER CELL

# word_count
df["word_count"] = df["message"].astype(str).apply(lambda x: len(x.split()))

# char_count
df["char_count"] = df["message"].astype(str).apply(len)

# num_count (digits 0–9)
df["num_count"] = df["message"].astype(str).str.count(r"\d")

# upper_word_count (ALL-CAPS words)
def count_upper_words(text):
    return sum(1 for word in str(text).split() if word.isupper())

df["upper_word_count"] = df["message"].apply(count_upper_words)

# Display full DataFrame
print(df)

   id                                            message  word_count  \
0   1  Hi!! My ORDER is late :(  Order# ORD-1042. Ema...          12   
1   2  Refund please!!! I was charged 2 times... invo...          11   
2   3        Great service, thanks! arrived in 2 days :)           8   
3   4  WHY is my package DAMAGED??? tracking says del...           8   
4   5  Need to change address: 7421 Frankford Rd Apt ...          12   
5   6  Support ticket: ORD-1050. Call me at (469) 555...           9   
6   7  I can’t login— password reset link not working...          10   
7   8  Item missing from box. pls send replacement!! ...          10   

   char_count  num_count  upper_word_count  
0          73          4                 2  
1          71          9                 3  
2          43          1                 0  
3          55          0                 2  
4          67         13                 1  
5          58         14                 2  
6          76          0            

## Q3 (3 points) — Answer below

In [10]:
# Q3 — ANSWER CELL

from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
STOPWORDS = set(ENGLISH_STOP_WORDS)

def clean_text(text: str) -> str:
    text = str(text)

    # 1. Lowercase
    text = text.lower()

    # 2. Remove punctuation/symbols (keep letters/numbers/spaces)
    text = re.sub(r"[^a-z0-9\s]", " ", text)

    # 3. Remove stopwords
    words = text.split()
    words = [word for word in words if word not in STOPWORDS]

    # 4. Remove extra spaces
    text = " ".join(words)

    return text.strip()

# Create clean column
df["clean"] = df["message"].apply(clean_text)

# Print original and clean for id=1 and id=4
print("ID=1")
print("Original:", df.loc[df["id"] == 1, "message"].values[0])
print("Clean:", df.loc[df["id"] == 1, "clean"].values[0])

print("\nID=4")
print("Original:", df.loc[df["id"] == 4, "message"].values[0])
print("Clean:", df.loc[df["id"] == 4, "clean"].values[0])

ID=1
Original: Hi!! My ORDER is late :(  Order# ORD-1042. Email me at sara.Ali@gmail.com
Clean: hi order late order ord 1042 email sara ali gmail com

ID=4
Original: WHY is my package DAMAGED??? tracking says delivered...
Clean: package damaged tracking says delivered


## Q4 (2 points) — Answer below

In [11]:
# Q4 — ANSWER CELL

# Extract order_id (case-insensitive)
df["order_id"] = df["message"].str.extract(r"(ORD-\d{4})", flags=re.IGNORECASE)

# Extract email
df["email"] = df["message"].str.extract(
    r"([A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,})"
)

# Display required columns
print(df[["id", "order_id", "email"]])

   id  order_id                  email
0   1  ORD-1042     sara.Ali@gmail.com
1   2  ORD-1042                    NaN
2   3       NaN                    NaN
3   4       NaN                    NaN
4   5       NaN                    NaN
5   6  ORD-1050                    NaN
6   7       NaN  mehri.sattari@unt.edu
7   8  ord-1060                    NaN


## Q5 (1 point) — Answer below

In [12]:
# Q5 — ANSWER CELL

from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

# Fit TF-IDF on clean column
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(df["clean"])

# Compute average TF-IDF per term
avg_tfidf = np.mean(tfidf_matrix.toarray(), axis=0)

# Get feature names
terms = vectorizer.get_feature_names_out()

# Combine terms with scores
term_scores = list(zip(terms, avg_tfidf))

# Sort by score descending
term_scores = sorted(term_scores, key=lambda x: x[1], reverse=True)

# Print top 5 keywords
print("Top 5 TF-IDF Keywords:")
for term, score in term_scores[:5]:
    print(f"{term}: {round(score, 4)}")

Top 5 TF-IDF Keywords:
ord: 0.1384
order: 0.099
1042: 0.0709
email: 0.0658
arrived: 0.0559


## Grading Checklist
- Q1: correct load + prints  
- Q2: correct counts  
- Q3: cleaning follows the required order + prints for id=1 and id=4  
- Q4: regex extraction works (case-insensitive `ORD-####` and emails)  
- Q5: prints 5 keywords + their scores (rounding is fine)
